<a href="https://colab.research.google.com/github/vasist1987/HackerRank_Python/blob/master/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarization


### Importing all requisite libraries
#### Implementing BART summarizer<br> Bart uses seq2seq architecture Bi-directional BERT encoder and left to right GPT decoder.<br> Only contractions were removed apart from that no other pre-processing is required for this dataset.

In [19]:
import numpy as np
import pandas as pd 
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('all') # one time execution
import re
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
# from attention import AttentionLayer
import gdown
import zipfile
import os
!pip install contractions
import contractions
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
!pip install transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration
import torch


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

#### downloading data from the google drive location provided

In [2]:

url = 'https://drive.google.com/uc?id=1VGthRzHtBSIO182zMCMiqY-YV-D0mLLG'
output = 'test.zip'
gdown.download(url, output, quiet=False) 


Downloading...
From: https://drive.google.com/uc?id=1VGthRzHtBSIO182zMCMiqY-YV-D0mLLG
To: /content/test.zip
210MB [00:01, 132MB/s]


'test.zip'

### Function to handle text bigger than the length BART embedding<br> [Function Source](https://colab.research.google.com/drive/1_BgZXBEYPaHapZFTl7ks9RxTxtwQRt7S#scrollTo=yw-RBPYeg0Nm&line=1&uniqifier=1)

In [3]:

def summarize(long_text):
  model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
  tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

  # tokenize without truncation
  inputs_no_trunc = tokenizer(long_text, max_length=None, return_tensors='pt', truncation=False)

  # get batches of tokens corresponding to the exact model_max_length
  chunk_start = 0
  chunk_end = tokenizer.model_max_length  # == 1024 for Bart
  inputs_batch_lst = []
  while chunk_start <= len(inputs_no_trunc['input_ids'][0]):
      inputs_batch = inputs_no_trunc['input_ids'][0][chunk_start:chunk_end]  # get batch of n tokens
      inputs_batch = torch.unsqueeze(inputs_batch, 0)
      inputs_batch_lst.append(inputs_batch)
      chunk_start += tokenizer.model_max_length  # == 1024 for Bart
      chunk_end += tokenizer.model_max_length  # == 1024 for Bart

  # generate a summary on each batch
  summary_ids_lst = [model.generate(inputs, num_beams=4, max_length=100, early_stopping=True) for inputs in inputs_batch_lst]

  # decode the output and join into one string with one paragraph per summary batch
  summary_batch_lst = []
  for summary_id in summary_ids_lst:
      summary_batch = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_id]
      summary_batch_lst.append(summary_batch[0])
  summary_all = '\n'.join(summary_batch_lst)
  return(summary_all)


#### Training data provided checking how close the summarization is.<br> Entire source text was concenated ass one and the "@highlight" section which had the summary as one.

In [4]:

df_train=pd.DataFrame()
with zipfile.ZipFile('test.zip') as z:
  for filename in z.namelist()[0:50]:
    with z.open(filename) as f:
      dfs=[]
      if(filename.find("stories_text_summarization_dataset_train") != -1):
        for lines in f:
          # lines=lines.splitlines()
          lines=lines.decode("utf8").strip()
          dfs.append(lines)
        dfs=' '.join(dfs)
        if(len(dfs)>1):
          x,y = dfs.split('@highlight',1)
          x=x.replace("U.S","United States")
          y=y.replace("U.S","United States")
          y=y.replace('@highlight','')
          x=contractions.fix(x)
          y=contractions.fix(y)
          summarized = summarize(x)
          df_temp=pd.DataFrame([x,y,summarized]).T
          df_train=df_train.append(df_temp)
df_train.columns=['Text','Provided_Summary','Predict_Summary']

Token indices sequence length is longer than the specified maximum sequence length for this model (1069 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2042 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1163 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1507 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

In [8]:
#### Testing data summarizing for the entire dataset

In [5]:

df_test=pd.DataFrame()
with zipfile.ZipFile('test.zip') as z:
  for filename in z.namelist():
    with z.open(filename) as f:
      dfs=[]
      if(filename.find("stories_text_summarization_dataset_test") != -1):
        for lines in f:
          # lines=lines.splitlines()
          lines=lines.decode("utf8").strip()
          dfs.append(lines)
        dfs=' '.join(dfs)
        if(len(dfs)>1):
          x = dfs.split('@highlight',1)[0]
          x=x.replace("U.S","United States")
          x=contractions.fix(x)
          summarized = summarize(x)
          df_temp=pd.DataFrame([x,summarized]).T
          df_test=df_test.append(df_temp)

df_test.columns=['Text','Predict_Summary']

Token indices sequence length is longer than the specified maximum sequence length for this model (1289 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1230 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1123 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1576 > 1024). Running this sequence through the model will result in indexing errors


In [6]:
df_train.head()

,Text,Provided_Summary,Predict_Summary
0,"TEHRAN, Iran (CNN) -- Iranian security forces blamed a privately owned vehicle Thursday for allegedly running into protesters during Sunday's opposition protests on a Shiite Muslim holy day, accor...","Video's posters say it shows Iranian police driving into crowds of demonstrators Truck drives over person in video, which posters say was recorded Sunday Seven people were killed in anti-g...",Iranian security forces blame privately owned vehicle for allegedly running into protesters. At least seven people were killed Sunday as anti-government protests turned violent. Iranian government...
0,"New York (CNN) -- More than 1,000 people in New Jersey and New York, many of them adolescent Orthodox Jews, have been sickened with mumps since August, health authorities said Monday. Orange Coun...",Almost all the cases of mumps involve adolescent Orthodox Jews Outbreak traced to boy who went to United Kingdom during mumps outbreak Severe cases of mumps can lead to brain inflammation ...,"Orange County, New York, has confirmed 494 cases since early November. Almost all of those infected with the virus are of the Orthodox or Hasidic Jewish population. Mumps outbreak began at a summe..."
0,(CNN) -- John Isner of the United States and his French opponent Nicolas Mahut made tennis history at Wimbledon on Wednesday as they contested the longest grand slam singles match of all time -- ...,John Isner and Nicolas Mahut set world record in marathon tennis match at Wimbledon The match is the longest in terms of time at 10 hours and also games contested Isner and Mahut tied at 5...,John Isner and Nicolas Mahut play longest grand slam singles match of all time. Match called off at 59 games all in the fifth and deciding set after 10 hours of action on Court 18. Isner has also ...
0,"NEW DELHI, India (CNN) -- There was stone cold silence in the car, as the Kumars drove home. Salwan Montessori School in New Delhi received 2,500 applications -- for 150 spots -- for the 2009 sch...","More Indians moving to urban areas, hence longer waiting lists for nursery schools Some schools have places for less than 4 percent of applicants Chosen nursery school determines 14 years ...","Nursery school admission is key to a child's chance at a better life in India. Many students spend 14 years in one school, so where they go to nursery determines the next 14 years of their educati..."
0,"LONDON, England (CNN) -- A visit to the Eiffel Tower, the Statue of Liberty and the Pyramids is on the itinerary of most jet-setting travelers. Visiting the Eiffel Tower was voted the most overra...","Some of most iconic global tourist attractions are most disappointing to visit The Eiffel Tower, the Statue of Liberty and the Pyramids on the list The Treasury in the ancient city of Petr...","Eiffel Tower, Statue of Liberty and Pyramids voted most disappointing international sights. Big Ben, Buckingham Palace and Princess Diana Memorial Fountain voted most overrated. Alnwick Castle in ..."


In [7]:
df_test.head()

,Text,Predict_Summary
0,"Soon after I first came to visit China in the autumn of 1971, I saw a contingent of militia soldiers doing marching drills in Tiananmen Square. I was told they were rehearsing for the annual Natio...","Mao Zedong formally proclaimed the founding of the People's Republic on October 1, 1949. Mao proved to be good at fighting but poor at governing. He pushed sweeping socio-economic initiatives and ..."
0,"Rory McIlroy is off to a good start at the Scottish Open. he is hoping for a good finish, too, after missing the cut at the Irish Open. McIlroy shot a course record 7-under-par 64 at Royal Aberde...",Rory McIlroy shoots a course record 7-under-par 64 at the Scottish Open. Sweden's Kristoffer Broberg had earlier fired a 65 at Royal Aberdeen. Last year's British Open champion Phil Mickelson is t...
0,"The man suspected of trying to attack the Dutch royal family by crashing his car near their bus has died, Dutch police said Friday. A car is pictured after crashing into the crowd waiting for the...","Police search man's house but find no weapons, explosives. Five people, as well as the driver, were killed and 12 were wounded in the incident. Incident happened during the country's annual Queen'..."
0,"Twitter is on the verge of its initial public offering and everyone is knickers have been in a knot all week over the company's lack of management diversity -- that is, women. As The New York Time...",Women make up 6% of chief executives at the leading 100 tech companies. Most startups have all-male boards. Women held 16.6% of Fortune 500 board seats in 2012. Women and minorities have lost grou...
0,"The White House Wednesday said it was ""puzzled"" by a former spokesman's memoir in which he accuses the Bush administration of being mired in propaganda and political spin and at times playing loos...","NEW: White House ""puzzled"" by memoir by former spokesman Scott McClellan. NEW: Former White House adviser Karl Rove says book sounds like ""left-wing blogger"" NEW: Another former Bush aide-turned-c..."


In [66]:
df_test.to_csv('summarization.csv',index=True)
from google.colab import files
files.download('summarization.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
#@title
sentences = []
for s in df_train['Text']:
  sentences.append(sent_tokenize(s))
sentences_test = []
for s in df_test['Text']:
  sentences_test.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x]
sentences_test = [y for x in sentences_test for y in x]

clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences_test = pd.Series(sentences_test).str.replace("[^a-zA-Z]", " ")
clean_sentences_test = [s.lower() for s in clean_sentences_test]

def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences_test = [remove_stopwords(r.split()) for r in clean_sentences_test]
# ! wget http://nlp.stanford.edu/data/glove.6B.zip
# ! unzip glove*.zip
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

sentence_vectors_test = []
for i in clean_sentences_test:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors_test.append(v)
sim_mat = np.zeros([len(sentences), len(sentences)])
sim_mat1 = np.zeros([len(sentences_test), len(sentences_test)])


for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

for i in range(len(sentences_test)):
  for j in range(len(sentences_test)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors_test[i].reshape(1,100), sentence_vectors_test[j].reshape(1,100))[0,0]

nx_graph = nx.from_numpy_array(sim_mat)
nx_graph1 = nx.from_numpy_array(sim_mat1)
scores = nx.pagerank(nx_graph)
scores_test = nx.pagerank(nx_graph1)

ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
ranked_sentences_test = sorted(((scores[i],s) for i,s in enumerate(sentences_test)), reverse=True)
sn = 1

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])